In [3]:
import numpy as np
from math import sqrt
import ot

In [4]:
def gen_4mom_approx(_w_dim: int, _batch_size: int, _W: np.ndarray = None, _K: np.ndarray = None, _H: np.ndarray = None):

    _a_dim = int(_w_dim*(_w_dim - 1)//2)

    lst = []
    for k in range(_w_dim):
        for l in range(k+1, _w_dim):
            lst.append((k,l))

    if _W is None:
        __W = np.random.randn(_batch_size, _w_dim)
    else:
        __W = _W

    if _H is None:
        __H = sqrt(1/12) * np.random.randn(_batch_size, _w_dim)
    else:
        __H = _H

    if _K is None:
        __K = sqrt(1/720) * np.random.randn(_batch_size, _w_dim)
    else:
        __K = _K

    squared_K = np.square(__K)
    C = np.random.exponential(8/15, size=(_batch_size, _w_dim))

    p = 21130/25621
    c = sqrt(1/3) - 8/15

    ber = np.random.binomial(1, p = p, size=(_batch_size, _a_dim))
    uni = np.random.uniform(-sqrt(3), sqrt(3), size=(_batch_size, _a_dim))
    rademacher = np.ones(_a_dim) - 2* np.random.binomial(1, 0.5, size= (_batch_size, _a_dim))
    ksi = ber*uni + (1-ber)*rademacher
    #print(ksi)
    def sigma(i: int, j: int):
        return np.sqrt(3/28*(C[:,i] + c)*(C[:,j] + c) + 144/28*(squared_K[:,i] + squared_K[:,j]))

    idx = 0
    for k in range(_w_dim):
        for l in range(k+1, _w_dim):
            sig = sigma(k, l)
            #print(f"shape: {sig.shape}, k: {k}, l: {l}, sig: {sig}")

            # now calculate a from ksi and sigma (but store a in ksi)
            ksi[:, idx] *= sig

            # calculate the whole thing
            ksi[:, idx] += __H[:,k]*__W[:,l] - __W[:,k]*__H[:,l] + 12*(__K[:,k]*__H[:,l] - __H[:,k]*__K[:,l])
            idx += 1

    return ksi

print(gen_4mom_approx(3, 5))

[[ 0.08026749 -0.18760197 -0.83153504]
 [-0.19349161  0.97246473  0.71429675]
 [ 0.1669679  -0.06310642  0.2563318 ]
 [ 0.0818803   0.08882198  0.07325867]
 [-0.15384353  0.30631268 -1.14700493]]


In [5]:
samples = np.genfromtxt('samples/fixed_samples_3-dim.csv', dtype=np.float, delimiter=',')
W = samples[:,:3]
a_true = samples[:,3:6]
print(samples.shape)
bsz = samples.shape[0]
generated = gen_4mom_approx(3,bsz,_W=W)


/tmp/ipykernel_19802/1455709601.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  samples = np.genfromtxt('samples/fixed_samples_3-dim.csv', dtype=np.float, delimiter=',')


(65536, 6)


In [6]:
print(generated.shape)

(65536, 3)


In [7]:
err = [sqrt(ot.wasserstein_1d(a_true[:,i],generated[:,i],p=2)) for i in range(3)]
print(err)

[0.005665698090499184, 0.005825378048434068, 0.005386683451518001]


In [8]:
def four_combos(n: int):
    lst = []
    for i in range(n):
        for j in range(i,n):
            for k in range(j,n):
                for l in range(k,n):
                    lst.append((i,j,k,l))
    return lst

print(four_combos(3))

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 0, 2), (0, 0, 1, 1), (0, 0, 1, 2), (0, 0, 2, 2), (0, 1, 1, 1), (0, 1, 1, 2), (0, 1, 2, 2), (0, 2, 2, 2), (1, 1, 1, 1), (1, 1, 1, 2), (1, 1, 2, 2), (1, 2, 2, 2), (2, 2, 2, 2)]


In [12]:
def fourth_moments(input: np.ndarray):
    lst = four_combos(3)
    res = []
    for i,j,k,l in lst:
        col = input[:,i]*input[:,j]*input[:,k]*input[:,l]
        res.append(col.mean())
    return res


w2 = np.concatenate((W,W,W,W), axis=0)
print(w2.shape)
combo_list = four_combos(3)
moms = fourth_moments(generated)
moms2 = fourth_moments(a_true)
for i in range(len(combo_list)):
    print(f"combo: {combo_list[i]}, 4_match_RV moment: {moms[i] :.7f}, samples moment: {moms2[i] :.7f}")

combo: (0, 0, 0, 0), 4_match_RV moment: 0.1322672, samples moment: 0.1357276
combo: (0, 0, 0, 1), 4_match_RV moment: 0.0327312, samples moment: 0.0357896
combo: (0, 0, 0, 2), 4_match_RV moment: 0.0665591, samples moment: 0.0649001
combo: (0, 0, 1, 1), 4_match_RV moment: 0.0712622, samples moment: 0.0728633
combo: (0, 0, 1, 2), 4_match_RV moment: 0.0007492, samples moment: 0.0013768
combo: (0, 0, 2, 2), 4_match_RV moment: 0.0733880, samples moment: 0.0717662
combo: (0, 1, 1, 1), 4_match_RV moment: 0.0485160, samples moment: 0.0485544
combo: (0, 1, 1, 2), 4_match_RV moment: 0.0281755, samples moment: 0.0279942
combo: (0, 1, 2, 2), 4_match_RV moment: 0.0036255, samples moment: 0.0053913
combo: (0, 2, 2, 2), 4_match_RV moment: 0.0786498, samples moment: 0.0750950
combo: (1, 1, 1, 1), 4_match_RV moment: 0.2963159, samples moment: 0.2966960
combo: (1, 1, 1, 2), 4_match_RV moment: -0.0402266, samples moment: -0.0387191
combo: (1, 1, 2, 2), 4_match_RV moment: 0.0813640, samples moment: 0.08177

In [13]:
w2 = np.concatenate((W,W,W,W), axis=0)
print(w2.shape)

(262144, 3)


In [15]:
generated = gen_4mom_approx(3,4*bsz,_W=w2)
moms = fourth_moments(generated)
for i in range(len(combo_list)):
    print(f"combo: {combo_list[i]}, 4_match_RV moment: {moms[i] :.7f}, samples moment: {moms2[i] :.7f}")

combo: (0, 0, 0, 0), 4_match_RV moment: 0.1345208, samples moment: 0.1357276
combo: (0, 0, 0, 1), 4_match_RV moment: 0.0322235, samples moment: 0.0357896
combo: (0, 0, 0, 2), 4_match_RV moment: 0.0671989, samples moment: 0.0649001
combo: (0, 0, 1, 1), 4_match_RV moment: 0.0723331, samples moment: 0.0728633
combo: (0, 0, 1, 2), 4_match_RV moment: -0.0001302, samples moment: 0.0013768
combo: (0, 0, 2, 2), 4_match_RV moment: 0.0746322, samples moment: 0.0717662
combo: (0, 1, 1, 1), 4_match_RV moment: 0.0477691, samples moment: 0.0485544
combo: (0, 1, 1, 2), 4_match_RV moment: 0.0281756, samples moment: 0.0279942
combo: (0, 1, 2, 2), 4_match_RV moment: 0.0034754, samples moment: 0.0053913
combo: (0, 2, 2, 2), 4_match_RV moment: 0.0796670, samples moment: 0.0750950
combo: (1, 1, 1, 1), 4_match_RV moment: 0.2946939, samples moment: 0.2966960
combo: (1, 1, 1, 2), 4_match_RV moment: -0.0403792, samples moment: -0.0387191
combo: (1, 1, 2, 2), 4_match_RV moment: 0.0830674, samples moment: 0.0817